# Dog Breed Classification on 120 Breeds

## Import Keras and its related tools

In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


## Using InceptionV3 model with imagent pretrained weights

In [2]:
input_tensor = Input(shape=(350, 350, 3)) 
base_model = InceptionV3(input_tensor= input_tensor, weights='imagenet', include_top=True)
x = base_model.output
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(120, activation='softmax')(x)

## Using adam optimizer and tuned to achieve the accuracy

In [3]:
model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers:
    layer.trainable = False
adam = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics = ['accuracy'])

## Preprocessing the image and augment it by adding zoom, flip , zca whitening 

In [4]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.3,
        horizontal_flip=True,
        zca_whitening = True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'train/',
        target_size=(350, 350),
        batch_size=128,
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        'test/',
        target_size=(350, 350),
        batch_size=128,
        class_mode='categorical')


Found 9022 images belonging to 120 classes.
Found 1200 images belonging to 120 classes.


## Label map to map the breed name with its values

In [5]:
label_map = training_set.class_indices

{'affenpinscher': 0, 'afghan_hound': 1, 'african_hunting_dog': 2, 'airedale': 3, 'american_staffordshire_terrier': 4, 'appenzeller': 5, 'australian_terrier': 6, 'basenji': 7, 'basset': 8, 'beagle': 9, 'bedlington_terrier': 10, 'bernese_mountain_dog': 11, 'black-and-tan_coonhound': 12, 'blenheim_spaniel': 13, 'bloodhound': 14, 'bluetick': 15, 'border_collie': 16, 'border_terrier': 17, 'borzoi': 18, 'boston_bull': 19, 'bouvier_des_flandres': 20, 'boxer': 21, 'brabancon_griffon': 22, 'briard': 23, 'brittany_spaniel': 24, 'bull_mastiff': 25, 'cairn': 26, 'cardigan': 27, 'chesapeake_bay_retriever': 28, 'chihuahua': 29, 'chow': 30, 'clumber': 31, 'cocker_spaniel': 32, 'collie': 33, 'curly-coated_retriever': 34, 'dandie_dinmont': 35, 'dhole': 36, 'dingo': 37, 'doberman': 38, 'english_foxhound': 39, 'english_setter': 40, 'english_springer': 41, 'entlebucher': 42, 'eskimo_dog': 43, 'flat-coated_retriever': 44, 'french_bulldog': 45, 'german_shepherd': 46, 'german_short-haired_pointer': 47, 'gian

## Start training the model

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5', verbose=1, save_best_only=True)
callbacks_list = [checkpointer]
model.fit_generator(
        training_set,
        steps_per_epoch=128,
        epochs=20,
        validation_data=test_set,
        callbacks=callbacks_list)

Epoch 1/50


/Users/sparshgarg/anaconda3/python/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:594: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/Users/sparshgarg/anaconda3/python/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:612: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


128/128 [==============================] - 13118s 102s/step - loss: 3.4071 - acc: 0.4711 - val_loss: 0.7774 - val_acc: 0.8958

Epoch 00001: val_loss improved from inf to 0.77745, saving model to weights-improvement-01-0.90.hdf5
Epoch 2/50
128/128 [==============================] - 13429s 105s/step - loss: 1.0377 - acc: 0.7969 - val_loss: 0.4151 - val_acc: 0.9067

Epoch 00002: val_loss improved from 0.77745 to 0.41512, saving model to weights-improvement-02-0.91.hdf5
Epoch 3/50
128/128 [==============================] - 9159s 72s/step - loss: 0.7856 - acc: 0.8259 - val_loss: 0.3916 - val_acc: 0.9117

Epoch 00003: val_loss improved from 0.41512 to 0.39161, saving model to weights-improvement-03-0.91.hdf5
Epoch 4/50
128/128 [==============================] - 8328s 65s/step - loss: 0.7196 - acc: 0.8334 - val_loss: 0.3803 - val_acc: 0.9033

Epoch 00004: val_loss improved from 0.39161 to 0.38030, saving model to weights-improvement-04-0.90.hdf5
Epoch 5/50
128/128 [===========================

## Training was taking time so moved to Google cloud and trained it and found after 13 epochs it was converged so stopped at 13 epoch

<img src="Result.png">

### Model is available here : https://www.dropbox.com/s/ljjrd5huf4smypg/weights-improvement-12-0.91.hdf5?dl=0